In [35]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
import os
import psycopg2
from dotenv import load_dotenv

# Cargar variables desde .env
load_dotenv()

DB_URL = os.getenv("DB_URL")

#######
# --- EMERGENCY RESET BUTTON ---

try:
    conn = psycopg2.connect(DB_URL)
    conn.autocommit = True
    with conn.cursor() as cur:
        kill_query = """
        SELECT pg_terminate_backend(pid)
        FROM pg_stat_activity
        WHERE datname = 'MyDatabase1_pickthrew'
            AND pid <> pg_backend_pid();
        """
        cur.execute(kill_query)
        print("All other connections closed!")
    conn.close()
except Exception as e:
    print(f"Could not reset: {e}")
########

conn = psycopg2.connect(DB_URL)


All other connections closed!


In [37]:
import pandas as pd

def display_table(query):
    cursor = conn.cursor()
    try:
        query = query
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        print("Error al mostrar la tabla:", e)
        return None
    finally:
        cursor.close()
        # NO cerramos conn aquí

def update_record(query, params=None):
    cursor = conn.cursor()
    try:
        cursor.execute(query, params)
        conn.commit()
        print("Record updated successfully")
    except Exception as e:
        conn.rollback()
        print("Error updating record:", e)
    finally:
        cursor.close()


In [38]:
data1 = display_table("""                                        
SELECT *
FROM my_schema_1.reminders
WHERE activated = TRUE
"""
)

In [39]:
import requests

BOT_TOKEN = os.getenv("BOT_TOKEN")

def send_telegram_message(token, chat_id, text_msg):
    try:
        url = f"https://api.telegram.org/bot{token}/sendMessage"
        payload = {"chat_id": chat_id, "text": text_msg}
        response = requests.post(url, data=payload)
        return response.json()
    except Exception as e:
        print(f"⚠️ Error: {e}")
        return None

In [48]:
from datetime import datetime
import time
now = datetime.now()

now_yyyymm = now.strftime("%Y-%m")

print(now_yyyymm)

2026-01


In [ ]:
import os

import time
from datetime import datetime, time as dt_time 

now_hour_value = (now.hour - 3) if "GITHUB_REPOSITORY_OWNER_ID" in os.environ else now.hour

dt_time_now = dt_time(now_hour_value, now.minute)

print(f"dt_time_now: {dt_time_now}")
print(f"now_hour_value: {now_hour_value}")
print("\n")

for _, row in data1.iterrows():
    should_send = False
    frequency = row['frequency']

    day_of_week = row['day_of_week']
    day_value = row['day_value']
    month_value = row['month_value']
    year_value = row['year_value']
    hour_value = row['hour_value']
    minute_value = row['minute_value']
    last_completed_at = row['last_completed_at']
    

    
    last_completed_at_yyyymm = last_completed_at.strftime("%Y-%m")

    chat_id = row['chat_id']
    reminder = row['reminder']

    print(f"last_completed_at: {last_completed_at}")
    print(f"frequency: {frequency}")
    print(f"id: {row['id']}")

    # MONTHLY
    if (
        frequency == "monthly"
        and round(day_value) <= now.day
        and last_completed_at_yyyymm != now_yyyymm
    ):
        print(f"last_completed_at_yyyymm: {last_completed_at_yyyymm}")
        print(f"now_yyyymm: {now_yyyymm}")
        should_send = True

    # DAILY
    if frequency == "daily":
        time_value = dt_time(int(hour_value), int(minute_value))
        if time_value <= dt_time_now and (pd.isna(last_completed_at) or last_completed_at.date() != now.date()):

            print(f"time_value: {time_value}")
            should_send = True
    
    if should_send:
        send_telegram_message(BOT_TOKEN, chat_id, reminder)
        print("Sent")

        now = datetime.now()

        query123 = """
        UPDATE my_schema_1.reminders
        SET last_completed_at = %s
        WHERE id = %s
        """

        update_record(query123, (datetime(int(now.year),int(now.month),int(now.day),now_hour_value,int(now.minute)), row['id']))

    time.sleep(3)  # delays for 3 seconds
    print("\n")


dt_time_now: 14:00:00
now_hour_value: 14
last_completed_at: 2026-01-06 15:42:49.480696
frequency: monthly
id: 1


last_completed_at: 2026-01-06 15:42:53.429629
frequency: monthly
id: 3


last_completed_at: 2026-01-06 15:42:57.542781
frequency: monthly
id: 4


last_completed_at: 2026-01-06 13:35:00
frequency: daily
id: 9




In [8]:
# Cerrar conexión al final
conn.close()